In [93]:
pip install yahoofinancials

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 95, Finished, Available)

Note: you may need to restart the kernel to use updated packages.


In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from yahoofinancials import YahooFinancials
import datetime
import pprint
from pyspark.sql.functions import to_date

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 96, Finished, Available)

In [95]:
# print(YahooFinancials('AUDUSD=X').get_recommendations())

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 97, Finished, Available)

In [96]:
all_tickers = ["AAPL","MSFT","CSCO","AMZN","HDFCBANK.NS","BAJFINANCE.NS","META","NFLX","GOOG","TSLA","BTC-USD",
"CL=F","EURUSD=X","IDEA","PFC","VSTM", "ZS","KPTI","TRVN", "MRNS","VTVT", "VTGN", "AKAN", "CRKN", "BRSH","GWAV", 
"ETH-USD","DOGE-USD", "ADA-USD", "XRP-USD", "LTC-USD", "BNB-USD","USDT-USD", "LINK-USD", "MATIC-USD", 
"PEGY", "HSCS", "LUCY", "HPCO", "PXMD","ILAG","VRAX","HSCS","TGUSX", "TGUNX", "SSHQX", "QALGX", "TROCX", 
"FMEIX","FCPEX", "FLVEX", "FLCEX", "FLGEX", "FIENX", "ALGYX", "ALAFX", "ALZFX", "ALCFX", "ALGRX", "BIOUX", 
"CNRG", "ACES", "SMOG", "PBD", "KOMP", "HAIL", "EURHUF=X", "HUF=X", "GBPHUF=X", "OTP.BD", "CHFHUF=X", "RUB=X", 
"EURRUB=X", "TRY=X", "CNY=X", "RUBUSD=X", "EURCHF=X", "EURGBP=X", "EURJPY=X", "GBPCHF=X", "CHFEUR=X", "AUDUSD=X", 
  "WDAY","IGEN","BCG","NOW","OKTA","TEAM","CVAS","TWLO","NVGT","SWIN","CTNT","CCRDF","PFE","UCNQX", "NZDUSD=X", 
  "GBPUSD=X", "AUDJPY=X", "EURJPY=X", 
  "BDRX","FLJ", "AIRE","WBD", "JCI", "NKGN", "ONMD","MGOL","ZCAR","BDRX","MGRX","GWAV" ]
# all_tickers = ["BABA", "AAPL","MSFT","CSCO"]
# all_tickers = ["BABA"]

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 98, Finished, Available)

### To remove the duplicates

In [97]:
all_tickers = set(all_tickers)


StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 99, Finished, Available)

In [98]:
ticker_type = pd.DataFrame()
ticker_type.drop(ticker_type.index,inplace=True) 
ticker_type['Tickers'] = pd.DataFrame(all_tickers)
ticker_type['Type'] = ''

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 100, Finished, Available)

In [99]:
global ohlv_dict 
ohlv_dict = {}
ohlv_df = pd.DataFrame()
ohlv_df.drop(ohlv_df.index,inplace=True) 

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 101, Finished, Available)

ohlv_dict = {}
end_date = (datetime.date.today()).strftime('%Y-%m-%d')
beg_date = (datetime.date.today()-datetime.timedelta(1)).strftime('%Y-%m-%d')
for ticker in all_tickers:
    instr_type = ''
    yahoo_financials = YahooFinancials(ticker)
    json_obj = yahoo_financials.get_historical_price_data(beg_date,end_date,"daily")
    ohlv = json_obj[ticker]['prices']
    temp = pd.DataFrame(ohlv)[["formatted_date","adjclose","open","low","high","volume"]]
    temp.set_index("formatted_date",inplace=True)
    temp.dropna(inplace=True)
    ohlv_dict[ticker] = temp
    mask = ticker_type['Tickers'] == ticker
    instr_type = json_obj[ticker]['instrumentType']
    ticker_type.loc[mask, 'Type'] = instr_type    

In [100]:
def fn_fetch_type_data(beg_date, end_date, ticker):
    instr_type = ''
    yahoo_financials = YahooFinancials(ticker)
    json_obj = yahoo_financials.get_historical_price_data(beg_date,end_date,"daily")
    ohlv = json_obj[ticker]['prices']
    temp = pd.DataFrame(ohlv)[["formatted_date","adjclose","open","low","high","volume"]]
    temp.set_index("formatted_date",inplace=True)
    temp.dropna(inplace=True)
    ohlv_dict[ticker] = temp
    mask = ticker_type['Tickers'] == ticker
    instr_type = json_obj[ticker]['instrumentType']
    ticker_type.loc[mask, 'Type'] = instr_type 

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 102, Finished, Available)

In [101]:
global ohlv_dict 
ohlv_dict = {}
ohlv_df = pd.DataFrame()
ohlv_df.drop(ohlv_df.index,inplace=True) 
for ticker in all_tickers:
    yf = YahooFinancials(ticker)
    try:
        # global ohlv_dict = {}
        end_date = (datetime.date.today()).strftime('%Y-%m-%d')
        # end_date = (datetime.date.today()-datetime.timedelta(2)).strftime('%Y-%m-%d')
        # beg_date = (datetime.date.today()-datetime.timedelta(30)).strftime('%Y-%m-%d')
        beg_date = (datetime.date.today()-datetime.timedelta(1)).strftime('%Y-%m-%d')
        jo = yf.get_historical_price_data(beg_date,end_date,"daily")
        if ticker in jo and 'prices' in jo[ticker] and jo[ticker]['prices']:
            fn_fetch_type_data(beg_date,end_date,ticker)
            # fn_fetch_ohlv_data(ticker,ohlv_dict)
            print(f"Data type fetched:")
        else:
            print("no data")
    except Exception as e:
        print("error")

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 103, Finished, Available)

Data type fetched:
error
Data type fetched:
Data type fetched:
error
Data type fetched:
error
Data type fetched:
Data type fetched:
error
error
Data type fetched:
error
Data type fetched:
Data type fetched:
error
Data type fetched:
Data type fetched:
error
error
error
error
error
error
error
error
error
error
error
Data type fetched:
error
error
Data type fetched:
error
Data type fetched:
error
error
error
error
Data type fetched:
error
error
error
error
Data type fetched:
error
error
error
error
error
Data type fetched:
error
error
Data type fetched:
error
Data type fetched:
error
error
error
Data type fetched:
error
error
Data type fetched:
error
error
Data type fetched:
error
error
Data type fetched:
error
Data type fetched:
error
error
error
error
error
error
error
Data type fetched:
error
error
error
error
Data type fetched:
error
error
error
Data type fetched:
error
error
error
error
Data type fetched:
Data type fetched:
error
error
error
error
Data type fetched:
error
error
erro

In [102]:
def fn_fetch_ohlv_data(ticker, ohlv_dict):
    global temp_df
    global ohlv_df
    temp_df = pd.DataFrame(ohlv_dict[ticker])
    temp_df['DDate'] = temp_df.index
    temp_df['ticker'] = ticker
    temp_df = temp_df.reset_index(drop=True)
    ohlv_df = pd.concat([ohlv_df, temp_df], ignore_index=True, sort=False)
    temp_df.drop(temp_df.index,inplace=True) 

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 104, Finished, Available)

In [103]:
# ohlv_dict
# ohlv_df = pd.DataFrame()

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 105, Finished, Available)

In [104]:
ohlv_df = pd.DataFrame()
ohlv_df.drop(ohlv_df.index,inplace=True) 
for ticker in all_tickers:
    yf = YahooFinancials(ticker)
    try:
        end_date = (datetime.date.today()).strftime('%Y-%m-%d')
        # end_date = (datetime.date.today()-datetime.timedelta(1)).strftime('%Y-%m-%d')
        # beg_date = (datetime.date.today()-datetime.timedelta(30)).strftime('%Y-%m-%d')
        beg_date = (datetime.date.today()-datetime.timedelta(1)).strftime('%Y-%m-%d')
        jo = yf.get_historical_price_data(beg_date,end_date,"daily")
        if ticker in jo and 'prices' in jo[ticker] and jo[ticker]['prices']:
            # fn_fetch_type_data(beg_date,end_date,ticker)
            fn_fetch_ohlv_data(ticker, ohlv_dict)
            print(f"Data fetched:")
        else:
            print("no data")
    except Exception as e:
        print("error")

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 106, Finished, Available)

Data fetched:
error
Data fetched:
Data fetched:
error
Data fetched:
error
Data fetched:
Data fetched:
error
error
Data fetched:
error
Data fetched:
Data fetched:
error
Data fetched:
Data fetched:
error
error
error
error
error
error
error
error
error
error
error
Data fetched:
error
error
Data fetched:
error
Data fetched:
error
error
error
error
Data fetched:
error
error
error
error
Data fetched:
error
error
error
error
error
Data fetched:
error
error
Data fetched:
error
Data fetched:
error
error
error
Data fetched:
error
error
Data fetched:
error
error
Data fetched:
error
error
Data fetched:
error
Data fetched:
error
error
error
error
error
error
error
Data fetched:
error
error
error
error
Data fetched:
error
error
error
Data fetched:
error
error
error
error
Data fetched:
Data fetched:
error
error
error
error
Data fetched:
error
error
error
error
Data fetched:
error
Data fetched:
error
error


In [105]:
# ohlv_df.rename(columns = {'Date':'DDate'}, inplace = True)

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 107, Finished, Available)

In [106]:
new_order = ['DDate', 'ticker', 'adjclose', 'open', 'low', 'high','volume']
ohlv_df = ohlv_df[new_order]
# ohlv_df

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 108, Finished, Available)

In [107]:
ohlv_df.info()

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 109, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DDate     61 non-null     object 
 1   ticker    61 non-null     object 
 2   adjclose  61 non-null     float64
 3   open      61 non-null     float64
 4   low       61 non-null     float64
 5   high      61 non-null     float64
 6   volume    61 non-null     int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 3.5+ KB


In [108]:
ohlv_df['adjclose'] = ohlv_df['adjclose'].astype(float)
ohlv_df['open'] = ohlv_df['open'].astype(float)
ohlv_df['low'] = ohlv_df['low'].astype(float)
ohlv_df['high'] = ohlv_df['high'].astype(float)
ohlv_df['volume'] = ohlv_df['volume'].astype(float)

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 110, Finished, Available)

In [109]:
ohlv_df.dtypes

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 111, Finished, Available)

DDate        object
ticker       object
adjclose    float64
open        float64
low         float64
high        float64
volume      float64
dtype: object

In [110]:
ticker_tbl_sdf = spark.sql("SELECT * FROM YFinanceLH.ticker_type")
tt_sdf = spark.createDataFrame(ticker_type)
tt_v1_sdf = tt_sdf.withColumnRenamed("Type", "Type")


StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 112, Finished, Available)

In [111]:
output = tt_v1_sdf.join(ticker_tbl_sdf,['Tickers'],how='leftanti')

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 113, Finished, Available)

In [112]:
# display(output)

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 114, Finished, Available)

In [113]:
ticker_type_sdf  = output

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 115, Finished, Available)

In [114]:
# ticker_type_sdf = spark.createDataFrame(ticker_type)
ohlv_sdf = spark.createDataFrame(ohlv_df)


StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 116, Finished, Available)

In [115]:
ohlv_sdf = ohlv_sdf.withColumn("DDate", to_date(ohlv_sdf["DDate"], "yyyy-MM-dd"))

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 117, Finished, Available)

In [119]:
ticker_type_delta_table_path = "Tables/ticker_type"
ohlv_prices_delta_table_path = "Tables/ohlv_prices"
ticker_type_sdf.write.format("delta").mode("append").save(ticker_type_delta_table_path)
ohlv_sdf.write.format("delta").mode("append").save(ohlv_prices_delta_table_path)

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 121, Finished, Available)

In [117]:
ticker_type_sdf.drop() 
ohlv_sdf.drop() 

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 119, Finished, Available)

StatementMeta(, 5ad2f5c2-0f2b-4f77-80f4-c465804c3415, 122, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>